In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re
import io
import time
import string
import random

# https://criticalrole.fandom.com/wiki/Category:Transcripts
# https://criticalrole.fandom.com/wiki/Category:Transcripts?from=Narrative+Telephone%2FTranscripts

In [3]:
# df = pd.read_csv('../data/gutenberg_metadata.csv')
df = pd.read_json('../data/gutenberg-metadata.json').transpose()
df.head()

,author,formaturi,language,rights,subject,title
1,"[Jefferson, Thomas]","[https://www.gutenberg.org/files/1/1-0.txt, ht...",[en],[Public domain in the USA.],"[United States -- History -- Revolution, 1775-...",[The Declaration of Independence of the United...
2,[United States],[https://www.gutenberg.org/cache/epub/2/pg2.co...,[en],[Public domain in the USA.],"[JK, KF, Civil rights -- United States -- Sour...",[The United States Bill of Rights\r\nThe Ten O...
3,"[Kennedy, John F. (John Fitzgerald)]",[https://www.gutenberg.org/ebooks/3.html.image...,[en],[Public domain in the USA.],[Presidents -- United States -- Inaugural addr...,[John F. Kennedy's Inaugural Address]
4,"[Lincoln, Abraham]",[https://www.gutenberg.org/cache/epub/4/pg4.co...,[en],[Public domain in the USA.],"[Soldiers' National Cemetery (Gettysburg, Pa.)...",[Lincoln's Gettysburg Address\r\nGiven Novembe...
5,[United States],[https://www.gutenberg.org/cache/epub/5/pg5.co...,[en],[Public domain in the USA.],[United States -- Politics and government -- 1...,[The United States Constitution]


In [62]:

shelf = []
for index, row in df.iterrows():
    if (1 ==2 \
        # or 'fantasy' in str(row.subject).lower() \
        or 'adventure' in str(row.subject).lower() \
        # or  'science fiction' in str(row.subject).lower() \
        # or  'short stories' in str(row.subject).lower() \
        # or  'fairy tale' in str(row.subject).lower() \
        # or  'folklore' in str(row.subject).lower() \
        # or  'juvenile fiction' in str(row.subject).lower() \
        ) \
        and 'en' in str(row.language).lower() \
        and 'fiction' in str(row.subject).lower() \
        and 'rac' not in str(row.subject).lower() \
        and 'slave' not in str(row.subject).lower() \
        and 'sex' not in str(row.subject).lower() \
        and '.txt' in str(row.formaturi).lower() \
        and 'public domain in the usa' in str(row.rights).lower():

        shelf.append(row)

shelf

[author                                         [Hope, Anthony]
 formaturi    [https://www.gutenberg.org/ebooks/95.rdf, http...
 language                                                  [en]
 rights                             [Public domain in the USA.]
 subject      [Adventure stories, PR, Impostors and impostur...
 title                                  [The Prisoner of Zenda]
 Name: 95, dtype: object,
 author                                          [Verne, Jules]
 formaturi    [https://www.gutenberg.org/ebooks/103.rdf, htt...
 language                                                  [en]
 rights                             [Public domain in the USA.]
 subject      [Adventure stories, Voyages around the world -...
 title                        [Around the World in Eighty Days]
 Name: 103, dtype: object,
 author                                 [Burroughs, Edgar Rice]
 formaturi    [https://www.gutenberg.org/cache/epub/123/pg12...
 language                                          

In [63]:
new_df = pd.DataFrame(shelf)
# new_df.head()
# len(new_df)
# len(new_df.drop_duplicates('title'))
len(new_df.astype(str).drop_duplicates('title'))

1338

In [44]:
new_df['formaturi'].iloc[0]

['https://www.gutenberg.org/files/11/11-0.txt',
 'https://www.gutenberg.org/ebooks/11.epub.noimages',
 'https://www.gutenberg.org/ebooks/11.kindle.images',
 'https://www.gutenberg.org/cache/epub/11/pg11.cover.medium.jpg',
 'https://www.gutenberg.org/ebooks/11.epub.images',
 'https://www.gutenberg.org/files/11/11-h/11-h.htm',
 'https://www.gutenberg.org/files/11/11-0.zip',
 'https://www.gutenberg.org/ebooks/11.kindle.noimages',
 'https://www.gutenberg.org/ebooks/11.html.images',
 'https://www.gutenberg.org/ebooks/11.rdf',
 'https://www.gutenberg.org/files/11/11-h.zip',
 'https://www.gutenberg.org/cache/epub/11/pg11.cover.small.jpg']

In [3]:
df[df.Bookshelf=="Fantasy"]

,Title,Author,Link,Bookshelf
1123,The Wonderful Wizard of Oz,L. Frank Baum,http://www.gutenberg.org/ebooks/55,Fantasy
1124,Le Morte d'Arthur: Volume 1,Sir Thomas Malory,http://www.gutenberg.org/ebooks/1251,Fantasy
1125,The Legends of King Arthur and His Knights,Knowles and Malory,http://www.gutenberg.org/ebooks/12753,Fantasy
1126,The Marvelous Land of Oz,L. Frank Baum,http://www.gutenberg.org/ebooks/54,Fantasy
1127,The Mabinogion,Lady Charlotte Schreiber,http://www.gutenberg.org/ebooks/5160,Fantasy
1128,The Merry Adventures of Robin Hood,Howard Pyle,http://www.gutenberg.org/ebooks/964,Fantasy
1129,Le Morte d'Arthur: Volume 2,Sir Thomas Malory,http://www.gutenberg.org/ebooks/1252,Fantasy
1130,Irish Fairy Tales,James Stephens,http://www.gutenberg.org/ebooks/2892,Fantasy
1131,The House on the Borderland,William Hope Hodgson,http://www.gutenberg.org/ebooks/10002,Fantasy
1132,Dorothy and the Wizard in Oz,L. Frank Baum,http://www.gutenberg.org/ebooks/22566,Fantasy


In [17]:
df.Bookshelf.unique()

array(['Animal', 'Animals-Domestic', 'Animals-Wild',
       'Animals-Wild-Trapping', 'Animals-Wild-Birds',
       'Animals-Wild-Insects', 'Animals-Wild-Mammals',
       'Animals-Wild-Reptiles', 'Canada', "Children's", 'School',
       "Child's", 'Harvard', 'Classical', '6', 'Travel', 'Christmas',
       'Italy', 'United', 'Norway', 'Bulgaria', 'Argentina', 'South',
       "Women's", 'New', 'France', 'Greece', 'Germany', 'Australia',
       'Egypt', 'India', 'Africa', 'Crime', 'Mystery', 'Detective',
       'Language', 'Education', 'General', 'Romantic', 'FR', 'Western',
       'Erotic', 'Fantasy', 'Humor', 'Adventure', 'Science', 'Precursors',
       'Horror', 'Historical', 'Gothic', 'The', 'Architecture', 'Art',
       'Reference', 'IT', 'Czech', 'Maps', 'Camping', 'CIA',
       'Anthropology', 'Folklore', 'Current', 'Natural', 'Archaeology',
       'Biographies', 'German', 'Canon', 'Noteworthy', 'British', 'Music',
       'Opera', 'Bibliomania', 'Witchcraft', 'Psychology', 'Philosoph

In [24]:
genres = ["Classical", "Mystery", "Western", "Fantasy", "Adventure", "Gothic", "Folklore"]

df[df.Bookshelf.isin(genres)].groupby("Bookshelf").count()

,Title,Author,Link
Bookshelf,,,
Adventure,25,25,25
Classical,25,25,25
Fantasy,25,24,25
Folklore,25,25,25
Gothic,25,24,25
Mystery,13,13,13
Western,25,24,25


In [27]:
print(len(df[df.Bookshelf.isin(genres)].Title))
print(len(df[df.Bookshelf.isin(genres)].Title.unique()))


163
154


#### Test download with Frankenstein

In [44]:
link = 'http://www.gutenberg.org/ebooks/32947'
book_code = link.split('/')[-1]
book_code

'32947'

In [45]:
# https://www.gutenberg.org/cache/epub/42324/pg42324.txt

# book_code = 42324

url = f'https://www.gutenberg.org/cache/epub/{book_code}/pg{book_code}.txt'

html = requests.get(url).text

In [47]:
print(''.join(html.split("***")[2:])[200:700])

oogle Print project.)









[Illustration]




 ON

 SNAKE-POISON.

 ITS ACTION

 AND

 ITS ANTIDOTE.




 BY

 A. MUELLER, M.D.




 SYDNEY:

 L. BRUCK, MEDICAL PUBLISHER,
 13 CASTLEREAGH STREET.
 1893.




 SYDNEY:
 WEBDALE, SHOOSMITH & CO., PRINTERS,
 117 CLARENCE STREET.

 1893.




PREFACE.


Since the method of treating snakebite-poisoning by hypodermic
injections of strychnine, discovered by the writer and published but a
few ye


In [33]:
print(html[:10000])

The Project Gutenberg EBook of Frankenstein, by Mary W. Shelley

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Frankenstein
       or, The Modern Prometheus

Author: Mary W. Shelley

Release Date: March 13, 2013 [EBook #42324]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK FRANKENSTEIN ***




Produced by Greg Weeks, Mary Meehan and the Online
Distributed Proofreading Team at http://www.pgdp.net









                             FRANKENSTEIN:

                                  OR,

                         THE MODERN PROMETHEUS.

                          BY MARY W. SHELLEY.

            AUTHOR OF THE LAST MAN, PERKIN WARBECK, &c. &c.

    [Transcriber's Note: This text was produced from a photo-reprint of
    the 1831 edition.]


    REVISED, CORRECTED,
   

In [9]:
print(html[4500:5000])

f judge how far I possessed the
promise of better things hereafter. Still I did nothing. Travelling, and
the cares of a family, occupied my time; and study, in the way of
reading, or improving my ideas in communication with his far more
cultivated mind, was all of literary employment that engaged my
attention.

In the summer of 1816, we visited Switzerland, and became the neighbours
of Lord Byron. At first we spent our pleasant hours on the lake, or
wandering on its shores; and Lord Byr


##### Test unsuccessful

In [11]:
# https://www.gutenberg.org/cache/epub/42324/pg42l24.txt

url = f'https://www.gutenberg.org/cache/epub/42324/pg42l24.txt'

req = requests.get(url)
req.status_code

<Response [404]>

In [14]:
req.status_code

404

In [8]:
for index, row in df.iterrows():
    print(row['Title'])
    time.sleep(random.random()+0.5)

The Extermination of the American Bison
Deadfalls and Snares
Artistic Anatomy of Animals
Birds, Illustrated
On Snake-Poison: Its Action and Its Antidote
Fifty Years a Hunter and Trapper
What Bird is That?
Fox Trapping: A Book of Instruction Telling How to Trap, Snare, Poison and Shoot
A Guide for the Study of Animals
Our Vanishing Wild Life: Its Extermination and Preservation
Wolf and Coyote Trapping: An Up-to-Date Wolf Hunter's Guide


KeyboardInterrupt: 

In [31]:
def save_text(url, filename, dir='data/gutenberg/files/'):
    try:
        req = requests.get(url)
        if req.status_code == 200:
            soup = BeautifulSoup(req.text)

            with open(dir + filename +'.txt', 'w') as file:
                file.write(''.join(soup.text.split("***")[2:]))
            return filename
    except:
        return f'Skipped {filename}'

def get_book_link(url):
    book_code = url.split('/')[-1]
    url = f'https://www.gutenberg.org/cache/epub/{book_code}/pg{book_code}.txt'
    return url

# Main Loop

In [34]:
def main():
    df = pd.read_csv('data/gutenberg_metadata.csv')
    genres = ["Classical", "Mystery", "Western", "Fantasy", "Adventure", "Gothic", "Folklore"]

    df = df[df.Bookshelf.isin(genres)]
    i=0
    for index, row in df.iterrows():
        # print(row['Title']):
            # print(url)
            # episodes = get_episodes(url)
            filename = row['Title']
            url = get_book_link(row['Link'])
            # save_text(url,filename)
            # print(episodes)
            # for i in range(2):

            # filename = url.split('/')[-2]
            # url = get_transcript_link(url)
            save_text(url, filename=filename)
            print(f'{i}/{len(df)}:\t{filename}',end='\r',flush=True)
            time.sleep(random.random()+0.5)
            i+=1

    print(f'\Books Done.', flush=True)
        # print('\n ')

In [35]:
main()

\Books Done.hs and Legends of the Siouxa and the Old Southwestk-Lorelore: Old Timedda


In [20]:
row = df.iloc[5]

url = get_book_link(row['Link'])
# url
req = requests.get(url)
if req.status_code == 200:
    soup = BeautifulSoup(req.text)

            # with open(dir + filename +'.txt', 'w') as file:
            #     file.write(''.join(soup.split("***")[2:]))
            # return filename


In [30]:
# soup.text
''.join(soup.text.split("***")[2:])
# soup.text.split("***")

'\r\n\r\n\r\n\r\n\r\nProduced by Linda M. Everhart, Blairstown, Missouri\r\n\r\n\r\n\r\n\r\nFIFTY YEARS A HUNTER AND TRAPPER\r\n\r\n  [Frontispiece: E. N. WOODCOCK AND BEAR TRAPS--HIS OWN MAKE.]\r\n\r\nFIFTY YEARS A HUNTER AND TRAPPER\r\n\r\nExperiences and Observations of E. N. Woodcock\r\nthe noted Hunter and Trapper, as written\r\nby Himself and Published in\r\nH-T-T from 1903 to 1913\r\n\r\nEDITED BY\r\nA. R. HARDING\r\n\r\nPublished by\r\nA. R. HARDING, Publisher\r\nSt. Louis, Mo.\r\n\r\nCopyright 1913,\r\nBy\r\nA. R. HARDING.\r\n\r\n\r\n\r\nCONTENTS.\r\n\r\n  I--Autobiography of E. N. Woodcock\r\n  II--Early Experiences\r\n  III--My First Real Trapping Experience\r\n  IV--Some Early Experiences\r\n  V--Some Early Experiences (Concluded)\r\n  VI--A Hunt on the Kinzua\r\n  VII--My Last Hunt on the Kinzua\r\n  VIII--Fred and the Old Trapper\r\n  IX--Bears in 1870, Today--Other Notes\r\n  X--Incidents Connected with Bear Trapping\r\n  XI--Pacific Coast Trip\r\n  XII--Some Michigan Tr